# <center> Домашнее задание 4 </center>

## 1. Матрица Тёплица (30 баллов)

1. Напишите функцию, которая на вход принимает вектор уникальных значений {$a_{n-1},  \dots, a_{0}, \dots, a_{-(n+1)}$} и размер матрицы, а выдает сгенерированную на основании этих значений [матрицу Тёплица](https://en.wikipedia.org/wiki/Toeplitz_matrix). В решении нельзя использовать готовые реализации из ```scipy.linalg``` или других библиотек, но необходимо использовать ```numpy```. 
2. Напишите функцию, которая принимает на вход матрицу размера $N \times N$ и проверяет, является ли она матрицей Тёплица. Функция должна вернуть в ответе булевое значение (```True``` или ```False```). 

## 2. Ищем лучшую однушку на ЦИАНе (35 баллов)

В этом задании будем скачивать объявления о продаже однокомнатных квартир в Москве c [ЦИАНа](https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p=1&region=1&room1=1) и выберем самую понравившуюся из них! :3

1. Напишите функцию, которая парсит все объявления с одной страницы и по каждому из них сохраняет ссылку (href) на объявление с более подробной информацией. Примените эту функцию для первых 10 страниц, сохранив все href в одном списке. Если хотите собрать полную информацию и не боитесь быть забаненными, можете скачать все 54 страницы.

2. Напишите функцию, которая для каждой (href) страницы с объявлением квартиры качает и сохраняет: 
* **Общую стоимость**
* **Адрес**
* **Ближайшее метро**
* **Тип жилья**
* **Площадь комнат**
* **Этаж**
* **Срок сдачи**

Сохраните эту информацию в DataFrame.

3. Избавьтесь от лишних символов в колонках с ценой квартиры, этажом и площадью комнат, приведя их к числову типу. Определите пороги по этим переменным, ниже или выше которых вы будете считать это жилье неподходящим. Например, цена не более 15 миллионов, этаж не ниже 2, общая площадь не менее 40 $м^2$. Можете выбрать другие наиболее важные для вас критерии, всего их должно быть не меньше 3. 
Отфильтруйте DataFrame, оставив только те объявления, которые удовлетворяют условиям. Отсортируйте оставшиеся объявления по возрастанию цены и отобразить топ-10 финальных кандидатов. Понравилась какая-нибудь квартира из них? :) 

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# функция написана на основе примера с книгами, который мы разбирали на семинаре

def get_page(p):
    
    # делаем ссылку
    url='https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={}&region=1&room1=1'.format(p)
    
    # идем по ссылке
    response=requests.get(url)
    
    # делаем tree
    tree=BeautifulSoup(response.content, 'html.parser')
    
    # находим наши flats
    flats=tree.find_all('article', {'class':'_93444fe79c--container--2pFUD _93444fe79c--cont--1Ddh2'})
    
    
    # в словарь infa записываем данные по каждой книге
    infa=[]
    
    for flat in flats:
        infa.append({'price': flat.find('span', {'data-mark':'MainPrice', 'class':'_93444fe79c--text--2_SER'}).text[:-2].replace(' ', ''),
                     'address': flat.find('div', {'class':'_93444fe79c--labels--1J6M3'}).text,
                     'metro_station': flat.find('a', {'class':'_93444fe79c--link--3ruIo'}).text, 
                     })
        
        url_2=flat.a.get('href')
        response_2=requests.get(url_2)
        tree_2=BeautifulSoup(response_2.content, 'html.parser')
        
    return infa

In [3]:
infa=[]
for p in range(1, 10):
    infa.extend(get_page(p))

In [4]:
import pandas as pd

data=pd.DataFrame(infa)
print(data.shape)
data.head()

(252, 3)


,price,address,metro_station
0,21272737,"Москва, ЦАО, р-н Басманный, м. Бауманская, Бак...",Бауманская
1,32772547,"Москва, ЦАО, р-н Арбат, м. Арбатская, улица Но...",Арбатская
2,15900000,"Москва, СВАО, р-н Лосиноостровский, м. Бабушки...",Бабушкинская
3,55335000,"Москва, ЦАО, р-н Арбат, м. Смоленская, улица А...",Смоленская
4,10609435,"Москва, СЗАО, р-н Покровское-Стрешнево, м. Три...",Трикотажная


In [5]:
# меняем тип данных в колонке price
data['price'].astype(int)
data=data[data['price']<15000000]
data['price'].sor_values(ascending=False)

TypeError: '<' not supported between instances of 'str' and 'int'


## 3. Дефолты (35 баллов)

Судя по стоимости недвижимости в Москве, придется влезть в долги... Проанализируем, как часто люди не отдают ипотеки и кредиты! 

В файле ```defaults.csv``` предоставлены данные, которые можно использовать для анализа клиентской базы, предсказания дефолта, кредитного скоринга (т.е. для оценки кредитных рисков клиентов). 

Переменные:
* **ID** - идентификатор клиента;
* **LIMIT_BAL** - сумма предоставленного кредита/ипотеки (в долларах);
* **SEX** - пол (1=мужской, 2=женский);
* **EDUCATION** - уровень образования (1=научная степень, 2=высшее, 3=среднее, 4=другое, 5 и 6=неизвестно);
* **MARRIAGE** - семейное положение (1=женат/замужем, 2=холост/незамужняя, 3=другое);
* **AGE** - возраст (в годах);
* **PAY_0** - Статус погашения кредита в сентябре (-1 = надлежащая оплата, 1 = задержка платежа на один месяц, 2 = задержка платежа на два месяца, ... 8 = задержка платежа на восемь месяцев, 9 = задержка платежа на девять месяцев и более). **PAY_2, PAY_3, ..., PAY_6** - в августе, июле, июне, мае и апреле соответственно;
* **BILL_AMT1** - Сумма выписки по счету в сентябре (в долларах). **BILL_AMT2, BILL_AMT3, ..., BILL_AMT6** - в августе, июле, июне, мае и апреле соответственно;
* **PAY_AMT1** - Сумма предыдущего платежа в сентябре (в долларах). **PAY_AMT1, PAY_AMT2, ..., PAY_AMT6** - в августе, июле, июне, мае и апреле соответственно;
* **default.payment.next.month**: произошел дефолт (1=да, 0=нет).

Нужно проанализировать набор данных банка, выполнив следующие задания:
1. Загрузить данные и преобразуйте в pd.DataFrame. Какого размера этот датасет? Посмотрите на первые 10 строк ваших данных. С помощью ```random.randint()``` в диапазоне значений индексов строк сгенерируйте случайное число $N$ и верните строки из датафрейма по индексам от $N$ до $N + 20$.<br>
2. Отобразите количество пропущенных значений во всех колонках.  <br>
3. Выведите количество уникальных клиентов в датасете, покажите не менее 2 разных способов это сделать. <br>
4. Проверьте, есть ли среди клиентов дисбаланс в количестве женщин и мужчин? Какого их соотношение (в процентах)? <br>
5. Создайте новую переменную **AGE_BINS** с категориями по переменной возраст (= возрастными диапазонами). Постройте и сравните графики распределения клиентов по значениям переменных *AGE* и *AGE_BINS*. Клиенты каких возрастов чаще всего приходят за кредитом в этот банк? Есть ли преимущество использования переменной c категориям? <br>
6. Выведите общее количество клиентов с дефолтом и без него. Постройте гистограмму с процентным отношением этих групп. Как можно оценить, велика ли полученная доля клиентов с дефолтом? <br>
7. Постройте гистограмму и график плотности распределения по суммам предоставленных кредитов (*LIMIT_BAL*) для каждой из групп клиентов отдельно (с дефолтом и без), поместив их на один график. Сделайте размер графика побольше, поменяйте дефолтные цвета. 
8. На основе графика из предыдущего пункта, ответьте на вопросы:
* По кредитам какого размера чаще всего происходят дефолты?
* На основе графиков плотности распределения, начиная от какой суммы кредита клиентов без дефолта становится больше, чем клиентов с дефолтом? 
* Как по графику определить, есть ли такие одобренные суммы кредитов, по которым никогда не происходит дефолт? Выведите значения а) сумм кредитов, по которым всегда встречается дефолт, б) и те, по которым никогда не бывает дефолта. <br>
9. Постройте матрицу корреляций для всех переменных по статусу погашения кредита. Добавьте значения корреляций на график. Есть ли общий тренд по возрастанию/убыванию корреляции по времени (всем месяцам)? <br>
10. Напишите функцию, которая возвращает словарь, где ключ - это сумма одобренного кредита, а значение - это вложенный словарь. В нем по ключу ```ratio``` хранится отношение клиентов с дефолтом к тем, у кого его не было, с такой суммой кредитов, а по ключу ```num_clients```  - общее число клиентов с таким кредитом. Пример элемента словаря: ``` {50000.0 : {'ratio': 0.357, 'num_clients': 3365}```. Отсортируйте получившийся словарь по ключу ```ratio``` в убывающем порядке. У какого размера кредита наибольшая доля клиентов с дефолтом? А сколько в датасете клиентов, по которым посчитана эта доля? Можем ли мы сделать вывод, что в целом не стоит одобрять эту сумму для кредита новым клиентам? <br>

Важно! Каждый график должен содержать как минимум заголовок и подписанные оси. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 10, 7

df=pd.read_csv('./defaults.csv')

In [ ]:
# 3.1 
# размер 
print('Размер датасета:', df.shape)

In [ ]:
# 3.1
# первые 10 строк
df.head(10)

In [ ]:
# 3.1
# вывод строк от n до (n+20)
n=np.random.randint(0, (df.shape[0]-20))
df[n:(n+20)]

In [ ]:
# 3.2
# количество пропущенных значений во всех колонках
df.isna().sum()

In [ ]:
# 3.3
# количество уникальных клиентов
df['ID'].nunique()

In [ ]:
# 3.4 
# женщин около 60%, а мужчин - около 40%
df['SEX'].value_counts(normalize=True)

In [ ]:
# 3.5
# находим возрастной диапазон
print(df['AGE'].min(), df['AGE'].max())

In [ ]:
# 3.5
# делим на категории по возрасту
df['AGE_BIN']=df['AGE']
df['AGE_BIN']=df['AGE_BIN'].astype(str)

groups=['20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79']
result=[]

for i in df['AGE']:
    j=int(i//5)
    result.append(groups[j-4])
    
df['AGE_BIN']=result

graph=df['AGE_BIN'].value_counts(normalize=True).plot.bar(title='Распределение по возрастным группам');
graph.set_xlabel('Возрастные группы')
graph.set_ylabel('Обращения в банк');

# чаще всего за кредитом обращаются клиенты 25-29 лет

In [ ]:
# 3.5
graph2=df['AGE'].value_counts(normalize=True).plot.bar(title='Распределение по возрасту')
graph2.set_xlabel('Возраст')
graph2.set_ylabel('Обращения в банк');

# по этом графику заметно, что использование деления на категории удобно для представления данных
# график с категориями визуально приятнее выглядит и проще воспринимается

In [ ]:
# 3.6
# количество клиентов с дефолтом и без
df['default.payment.next.month'].value_counts()

In [ ]:
# 3.6
# дефолтов по кредитам более 20%
graph3=df['default.payment.next.month'].value_counts(normalize=True).plot.bar(title='Соотношение кол-ва клиентов с дефолтом и без')
graph3.set_xlabel('Наличие/отсутствие дефолта')
graph3.set_ylabel('% клиентов');


Постройте гистограмму и график плотности распределения по суммам предоставленных кредитов (LIMIT_BAL) для каждой из групп клиентов отдельно (с дефолтом и без), поместив их на один график. Сделайте размер графика побольше, поменяйте дефолтные цвета.

## *4. Друзья друзей (бонусное, 30 баллов) 

1. С помощью VK API скачайте id всех ваших друзей. Убедитесь, что у вас открытый аккаунт, иначе будет ошибка доступа, и ничего скачать не удастся. Если вы не хотите открывать аккаунт, можете выбрать другой (например, аккаунт какой-то известной личности). 
2. Напишите функцию, которая с помощью ```users.get``` и ```friends.get``` по каждому человеку из списка друзей скачивает имя, фамилию, пол, город, дату рождения, образование, количество друзей. Напишите исключение, если у друга закрытый аккаунт, чтобы не включать его в анализ и функция не ломалась. 
3. Постройте график распределения количества друзей по всем друзьям. Создайте переменную ```friends_count_cat```, которая на основе количество друзей принимает значения $1$ - если друзей мало, $2$ - средне, $3$ - много друзей. Определите диапазоны для этих значений на основе графика с распределением. Постройте гистограмму по новой переменной. 
4. Разбейте колонку с образованием на две колонки ```university``` и ```faculty```, в первой сохранив название университета, во второй - название факультета. Удалите исходную колонку с образованием. Сгруппируйте данные по полу и постройте для каждой из групп на одном графике распределения 1) по городам, 2) университету, 3) факультету. Какие наблюдаются различия по этим переменным между девушками и молодыми людьми? 

In [ ]:
# Your code here